# Few-shot Learning na Prática (In-Context Learning)

Este notebook demonstra, de forma simples, como **um modelo pré-treinado** pode aprender um **padrão de tarefa** apenas observando **alguns exemplos no próprio prompt**, sem qualquer re-treinamento.

Chamamos isso de **Few-shot Learning em contexto (in-context learning)**.

---

## O que você vai ver neste notebook

1. Como montar um **prompt com poucos exemplos** (few-shots).
2. Como o modelo **induz a regra** a partir desses exemplos.
3. Dois tipos de tarefa sem treinar nada:
   - Classificação de sentimento em texto.
   - Extração de informações (nome e idade) de frases em português.

> Observação: este notebook usa modelos da biblioteca **Hugging Face Transformers**.
> Na primeira execução, o modelo será baixado (é necessário acesso à internet).

## 1. Preparando o ambiente

Nesta célula, vamos:

- Verificar a versão do Python e das bibliotecas principais.
- Detectar se há GPU (**cuda**), **mps** (Apple Silicon) ou apenas CPU.

Isso não altera o conceito de *few-shot learning*, mas ajuda a entender **onde** o modelo está sendo executado.

In [ ]:
import sys
import torch

print('Versão do Python:', sys.version.split()[0])
print('Versão do PyTorch:', torch.__version__)

device = torch.device(
    'cuda' if torch.cuda.is_available()
    else 'mps' if hasattr(torch, 'mps') and torch.mps.is_available()
    else 'cpu'
)
print('Dispositivo em uso:', device)

## 2. Carregando um modelo de linguagem (causal LM)

Usaremos um modelo pequeno da família **GPT-2** apenas para demonstração.

- Ele não é um modelo de 'chat', mas consegue **continuar textos**.
- Vamos explorar isso para mostrar o comportamento **few-shot**.

> Se você estiver em ambiente sem internet, a primeira execução pode falhar por não conseguir baixar o modelo.
> Ainda assim, leia o código e discuta o que ele faz.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = 'gpt2'
# model_name = "gpt2-medium"      # ou "gpt2-large", "gpt2-xl"

print('Carregando tokenizer e modelo:', model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Seleciona o device para o pipeline
if torch.cuda.is_available():
    dev = 0
elif hasattr(torch, 'mps') and torch.mps.is_available():
    # algumas versões podem não suportar 'mps' diretamente no pipeline;
    # se der erro, basta trocar para dev = -1 (CPU)
    dev = -1
else:
    dev = -1

text_gen = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=dev
)

print('Pipeline pronto.')

## 3. Exemplo 1 – Classificação de sentimento via Few-shot Learning

### Ideia

Vamos mostrar para o modelo alguns exemplos de frases com sentimentos **POSITIVO**, **NEGATIVO** e **NEUTRO**.

Depois, pedimos para ele **continuar o padrão** e classificar uma nova frase.

Não estamos treinando pesos.
Estamos apenas **mostrando o padrão no texto** e deixando o modelo *imitar*.

In [ ]:
prompt = '''
Você é um classificador de sentimento.
Como saída, use apenas as palavras: POSITIVO, NEGATIVO ou NEUTRO.

Exemplos:
Frase: "O filme foi maravilhoso, adorei cada cena." -> POSITIVO
Frase: "Estou muito decepcionado com o serviço." -> NEGATIVO
Frase: "A entrega ocorreu como esperado." -> NEUTRO

Agora classifique a frase a seguir como POSITIVO, NEGATIVO ou NEUTRO:
Frase: "O produto superou minhas expectativas!" ->
'''

saida = text_gen(
    prompt,
    max_new_tokens=10,
    temperature=0.1,  # menor temperatura = comportamento mais previsível
    top_p=0.9
)

print(saida[0]['generated_text'])

### Discussão

- O modelo **não foi treinado** especificamente para classificação de sentimento.
- Mesmo assim, ao ver **poucos exemplos** (few-shots), ele tenta **estender o padrão**.
- O resultado pode não ser perfeito, mas ilustra bem a ideia de:

> **Aprender pela configuração do prompt**, e não pela atualização de pesos.

Experimente alterar:

- As frases de exemplo.
- A frase a ser classificada.
- A instrução ('Use apenas as palavras...').

Veja como isso impacta o comportamento.

## 4. Exemplo 2 – Extração de informação (nome e idade)

Agora, vamos usar few-shot learning para **extrair informações estruturadas** de frases em português.

### Tarefa

Dado um texto com alguém e sua idade, queremos extrair:

- `Nome:`
- `Idade:`

Novamente, sem treinar nada. Apenas **mostrando exemplos no prompt**.

In [ ]:
prompt_ie = '''
Extraia o nome e a idade das frases a seguir.

Exemplos:
"Maria tem 32 anos." -> Nome: Maria | Idade: 32
"João completou 45 anos ontem." -> Nome: João | Idade: 45

Agora extraia os dados da frase a seguir e apresente no mesmo formato dos exemplos:
"Carla fará 29 anos no próximo mês." ->
'''

saida_ie = text_gen(
    prompt_ie,
    max_new_tokens=20,
    temperature=0.2,
    top_p=0.9
)

print(saida_ie[0]['generated_text'])

### Discussão

Observe que:

- O modelo tenta **seguir o formato** dos exemplos.
- Ele infere qualitativamente o que é **Nome** e o que é **Idade**.
- Ele não possui uma 'regra de idade' programada, mas **induz o padrão** a partir do texto.

Experimente:

- Alterar os exemplos (por exemplo, adicionar mais um caso).
- Mudar a forma do output (por exemplo, `JSON: {"nome": ..., "idade": ...}`).
- Traduzir tudo para inglês e comparar o comportamento.

## 5. Few-shot Learning x Fine-tuning (reflexão final)

Com este notebook, você viu que:

- Em **few-shot in-context learning**, o modelo não é re-treinado.
  - Não há *backpropagation* nem atualização de pesos.
  - O comportamento muda apenas pela **forma como construímos o prompt**.

- Em **fine-tuning**, há um processo explícito de treino:
  - Atualizamos pesos do modelo com gradiente.
  - O modelo 'incorpora' a nova tarefa de modo permanente.

### Perguntas para reflexão

1. Em que cenários few-shot learning é suficiente?
2. Quando seria necessário partir para fine-tuning?
3. Que vantagens e limitações você observou ao apenas mudar o prompt?

Use estas perguntas como base para sua participação em fórum ou relatório da disciplina.